<a href="https://colab.research.google.com/github/nickallen0/NOVA_takehome/blob/main/take_home_NOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import shutil
import glob
import random

# Mount Google Drive
drive.mount('/content/drive')


my_drive = "/content/drive/MyDrive"
root_folder = my_drive+'/NOVA_course_deep_learning_full/data/annotated_data/'  # Replace with the path to your root folder
main_folder = root_folder+'/full'  # Replace with the path to your main folder



Mounted at /content/drive


##Assemble all the class data into one unified folder structure and perform the train/val split

In [ ]:
# Create the folder structure if it doesn't exist for all the images and annotations
if not os.path.exists(main_folder):
    os.makedirs(main_folder)

if not os.path.exists(main_folder+'/images'):
    os.makedirs(main_folder+'/images')
images = main_folder+'/images'

if not os.path.exists(main_folder+'/labels'):
    os.makedirs(main_folder+'/labels')
labels = main_folder+'/labels'



In [ ]:
# Find all image files in the subfolders
image_files = glob.glob(os.path.join(root_folder, '*', '*', '*.tif'))
label_files = glob.glob(os.path.join(root_folder, '*', '*', '*.txt'))

# Copy image files to the main folder under images
for src in image_files:
    dst = os.path.join(images, os.path.basename(src))
    shutil.copy(src, dst)

# Copy label files into the main folder labels under labels
for src in label_files:
    dst = os.path.join(labels, os.path.basename(src))
    shutil.copy(src, dst)

In [ ]:
# split all the data into a train/val
path_to_tiles="/content/drive/MyDrive/NOVA_course_deep_learning_full/data/annotated_data/full"


In [ ]:
# define split for training and validation
split_train= 0.7
split_val=1-split_train

In [ ]:
# create train and val directories
train_dir = os.path.join(path_to_tiles, "train")
os.makedirs(train_dir, exist_ok=True) # creates new directory for training data
val_dir = os.path.join(path_to_tiles, "val")
os.makedirs(val_dir, exist_ok=True) # creates new directory for validation data
val_img_dir = os.path.join(path_to_tiles, "val","images")
os.makedirs(val_img_dir, exist_ok=True) # creates new directory for training data
train_img_dir = os.path.join(path_to_tiles, "train","images")
os.makedirs(train_img_dir, exist_ok=True) # creates new directory for training data
val_ann_dir = os.path.join(path_to_tiles, "val","labels")
os.makedirs(val_ann_dir, exist_ok=True) # creates new directory for training data
train_ann_dir = os.path.join(path_to_tiles, "train","labels")
os.makedirs(train_ann_dir, exist_ok=True) # creates new directory for training data

In [ ]:
# Get a list of all the .txt files in the data directory
txt_files = [f for f in os.listdir(labels) if f.endswith(".txt")]
img_files = [f for f in os.listdir(images) if f.endswith(".tif")]

In [ ]:
# Shuffle the list of text files
random.shuffle(txt_files)
#train=random.sample(txt_files, )

# Calculate the number of files for the train and validation sets
train_size = int(0.7 * len(txt_files))
val_size = len(txt_files) - train_size


In [ ]:
len(txt_files)

0

In [ ]:
# iterate through each annotated .txt file
for i, txt_file in enumerate(txt_files):
    if i < train_size:
        dest_dir = train_dir
    else:
        dest_dir = val_dir

    src_file = os.path.join(labels, txt_file)
    src_img = os.path.join(images, os.path.splitext(txt_file)[0] + ".tif")

    dest_file = os.path.join(dest_dir,"labels", txt_file)
    dest_img = os.path.join(dest_dir,"images", os.path.splitext(txt_file)[0]+".tif")

    #print("copying files")

    shutil.move(src_file, dest_file)
    shutil.move(src_img, dest_img)

# Train a model on the full class dataset

In [ ]:
%pip install comet_ml --quiet
import comet_ml # in future scripts this should be pasted before importing the ultralytics library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.7/534.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.


In [ ]:
# train a model
%pip install ultralytics
import ultralytics
ultralytics.checks()
import yaml

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
config_data = {
    'path': path_to_tiles,
    'train': path_to_tiles+'/train/images',
    'val': path_to_tiles+'/val/images',
    'test':"/content/drive/MyDrive/NOVA_course_deep_learning_full/data/annotated_data/test/images", # this is by default
    'names': {
        0: 'tree'
    }
}

with open(path_to_tiles+'/train_config.yaml', 'w') as file:
    yaml.dump(config_data, file)

In [ ]:
path_to_tiles+'/train_config.yaml'

'/content/drive/MyDrive/NOVA_course_deep_learning_full/data/annotated_data/full/train_config.yaml'

##Train

In [ ]:
!yolo train model=yolov8n.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=640 project=$path_to_tiles name="seedlingsFull_YOLOn_img640"

In [ ]:
!yolo train model=yolov8m.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=640 project=$path_to_tiles name="seedlingsFull_YOLOm_img640"

In [ ]:
!yolo train model=yolov8n.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1024 project=$path_to_tiles name="seedlingsFull_YOLOn_img1024"

In [ ]:
!yolo train model=yolov8m.pt data=$path_to_tiles'/train_config.yaml' epochs=300 imgsz=1024 project=$path_to_tiles name="seedlingsFull_YOLOm_img1024"

#Inference


In [ ]:
import os
annotator_ID=1 # change this to your folder ID

path_to_ortho_predict= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/orthomosaics/test_data/braatan_40m20230605_sun.tif" #now four of these not 1, will need to specify one by one
path_to_ortho_predict2= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/orthomosaics/test_data/galbyveien_20230504_sun.tif"
path_to_ortho_predict3= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/orthomosaics/test_data/krakstad_202304_sun.tif"
path_to_ortho_predict4= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/orthomosaics/test_data/ortho_hobol_042222_mavic_sun"

path_to_tiles_G="/content/drive/MyDrive/NOVA_course_deep_learning_full/data/tiles/10m_galbyveien_20230504_sun"
path_to_tiles_K="/content/drive/MyDrive/NOVA_course_deep_learning_full/data/tiles/10m_krakstad_202304_sun"
path_to_model="/content/drive/MyDrive/NOVA_course_deep_learning_full/data/annotated_data/full/seedlingsFull_YOLOn_img640/weights/best.pt"


In [ ]:
# other libraries
!pip install geopandas
!pip install rasterio
!pip install folium matplotlib mapclassify
# general python packages
import os, glob, shutil
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
#import warnings
#warnings.filterwarnings("ignore")
#import sys

# geospatial packages
from osgeo import gdal, ogr, osr
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio
path_osgeo_utils= "/usr/local/lib/python3.10/dist-packages/osgeo_utils" # defines path to gdal_retile.py

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Params:
#   - input_ortho_path: path to orthophoto to predict on ("/content/drive/MyDrive/NOVA_course_deep_learning/data/orthomosaics/test_data/krakstad_202304_sun.tif")
#   - footprints_path: path to multipolygon layer of tile footprints ("/content/drive/MyDrive/NOVA_course_deep_learning/data/map_data/drone_acquisitions.geojson")
#   - tile_size_m: tile size in m
#   - buffer_size_m= size of overlap area around each tile in m (used for removing boundary effects)
# input_ortho_path=
# footprints_path=

def tile_orthomosaic(input_ortho_path,footprints_path, tile_size_m, buffer_size_m):
  # Define path to data
  path_data="/content/drive/MyDrive/NOVA_course_deep_learning_full/data"

  # read drone acquisition footprints
  footprints= gpd.read_file(footprints_path)
  # Get ortho name
  ortho_name=os.path.splitext(os.path.basename(input_ortho_path)) [0]

  # create output dir
  output_tiles_dir=path_data+"/tiles/"+str(tile_size_m)+"m_"+ortho_name
  if not os.path.exists(output_tiles_dir):
    print("Creating output folder..."+ output_tiles_dir)
    os.makedirs(output_tiles_dir)

  # get raster metadata
  # Get pixel resolution (in meters) and tile size in pixels
  src_ds = gdal.Open(input_ortho_path) # reads in the orthomosaic
  _, xres, _, _, _, yres  = src_ds.GetGeoTransform() # get pixel size in meters
  print("Ortho resolution: "+str(round(xres,4))+" m")
  # Get EPSG code
  proj = osr.SpatialReference(wkt=src_ds.GetProjection())
  EPSG_code= proj.GetAttrValue('AUTHORITY',1)
  print("EPSG code: "+str(EPSG_code))
  # get number of bands
  n_bands=src_ds.RasterCount
  print("Number of bands: "+str(n_bands))

  # Compute tile and buffer size in pixels
  tile_size_px= round(tile_size_m/abs(xres)) # calculate the tile size in pixels
  buffer_size_px= round(buffer_size_m/abs(xres)) # calculate the buffer size in pixels
  tileIndex_name=ortho_name+"_tile_index" # define name for output tile index shapefile

  # Run gdal_retile.py using (can take some minutes)
  command_retile = "python "+path_osgeo_utils+"/gdal_retile.py -targetDir " + output_tiles_dir + " " + input_ortho_path+ " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of GTiff -tileIndex "+ tileIndex_name + " -tileIndexField ID"
  print(os.popen(command_retile).read()) ################################################################### SWITCH THIS ONE FOR TILING #########################################

  # cleanup tiles
  footprint_ortho= footprints[footprints['filename']==ortho_name]
  footprint_ortho_UU= footprint_ortho.geometry.unary_union
  # Load tiles shapefile
  tiles = gpd.read_file(output_tiles_dir+ "/"+ortho_name+"_tile_index.shp")
  tiles= tiles.to_crs(EPSG_code)

  # Select all tiles that are within the boundary polygon
  tiles_in = tiles[tiles.geometry.within(footprint_ortho_UU)]

  # Select all tiles that are not within the boundary polygon
  tiles_out= tiles.loc[~tiles['ID'].isin(tiles_in['ID']) ]
  print(str(len(tiles_out))+" tiles to be deleted")

  # delete tiles that are not within the footprint
  gtiffs_delete=[output_tiles_dir+ "/"+sub  for sub in tiles_out['ID']]
  for f in gtiffs_delete:
   if os.path.exists(f):
     os.remove(f)



In [ ]:
tile_orthomosaic(input_ortho_path=path_to_ortho_predict2, #need to iterate for the four areas in the folder
                 footprints_path= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/map_data/drone_acquisitions.geojson",
                 tile_size_m=10, # this could be changed
                 buffer_size_m=1) # this could be changed

In [ ]:
tile_orthomosaic(input_ortho_path=path_to_ortho_predict3, #need to iterate for the four areas in the folder
                 footprints_path= "/content/drive/MyDrive/NOVA_course_deep_learning_full/data/map_data/drone_acquisitions.geojson",
                 tile_size_m=10, # this could be changed
                 buffer_size_m=1) # this could be changed

In [ ]:
my_model = "/content/drive/MyDrive/NOVA_course_deep_learning/data/annotated_data/train/1/seedlingsHobol_YOLOm_img640/weights/best.pt"
full_model = path_to_model

In [ ]:
my_model

#Inference

###Use the model to find trees in the data

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
!yolo predict model=$my_model source=$path_to_tiles_K imgsz=640 conf=0.4 project=$path_to_tiles_K name=predict_tiles_mine save_txt=True save_conf=True save=True line_width=1

In [ ]:
!yolo predict model=$path_to_model source=$path_to_tiles_K imgsz=640 conf=0.4 project=$path_to_tiles_K name=predict_tiles_full save_txt=True save_conf=True save=True line_width=1

#Postprocessing

###Go from YOLO bounding boxes to domain metrics to compare the results

https://colab.research.google.com/drive/1LJh6ymvCOJbQoHOz1j61sMBTSf72BZaq?usp=sharing